# 1 Introducción
El siguiente cuaderno realiza la transpuesta de una matriz, utilizando GPGPU. El kernel ejecuta en hilos sobre dos dimensión (X e Y).

Para la resolución se aplicaron conceptos de Lenguaje Python, CUDA y el manejo de operaciones aritméticas sobre matrices.


---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo


In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   2#@param {type: "number"}

# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# CPU - Defino la memoria de la matriz.
M_cpu = numpy.random.randn( cantidad_N, cantidad_N)
M_cpu = M_cpu.astype( numpy.float32() )
T_cpu = numpy.random.randn( cantidad_N, cantidad_N)
T_cpu = T_cpu.astype( numpy.float32() )

tiempo_ini_cpu = datetime.now()

# CPU - reservo la memoria GPU.
M_gpu = cuda.mem_alloc( M_cpu.nbytes )
T_gpu = cuda.mem_alloc( T_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( M_gpu, M_cpu )
cuda.memcpy_htod( T_gpu, T_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void MatrizTranspuesta( int rows, int cols, float *mat_in, float *mat_out )
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x; 
    int idy = threadIdx.y + blockIdx.y * blockDim.y;
    if(idx < cols && idy < rows){
        int pos = idy * cols + idx;
        int trans_pos = idx * rows + idy;

        mat_out[trans_pos] = mat_in[pos];
    } 
}
""") 

# CPU - Genero la función kernel.
kernel = module.get_function("MatrizTranspuesta")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo_x = 16
dim_bloque_x = numpy.int( (cantidad_N+dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (cantidad_N+dim_hilo_y-1) / dim_hilo_y )

kernel( numpy.int32(cantidad_N), numpy.int32(cantidad_N), M_gpu, T_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )
tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( T_cpu, T_gpu )

#"""
# CPU - Informo el resutlado.
print( "------------------------------------")
print( "Matriz M: " )
print( M_cpu )
print( "------------------------------------")
print( "Transpuesta T: " )
print( T_cpu )
#"""

tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos N: ", cantidad_N )
print( "Cantidad de elementos M: ", cantidad_N )
print( "Thread x: ", dim_hilo_x, ", Bloque x:", dim_bloque_x )
print( "Thread y: ", dim_hilo_y, ", Bloque y:", dim_bloque_y )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )


---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa las matrices M y T
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria R a CPU memoria R.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones

A medida que aumenta la cantidad de elementos de la matriz aumenta el tiempo de procesamiento. A pesar de que en GPU tenemos más funciones y pasos para ejecutar (ej. reservar la memoria en area de GPU, copia de datos, definir y compilar Kernel, ejecución de Kernel) podemos notar que al realizar la transpuesta de la matriz de NxN elementos hacerlo con GPU reduce mucho el tiempo total comparado con el tiempo total de la misma operación solo con CPU. Esto ultimo se debe a que en GPU la ejecución se realiza mediante hilos concurrentes.

**Sugerencias de mejora:** 

1) Se podria realizar una mejora para que el algoritmo funcione para matrices no cuadradas. Ejemplo 
 N = 4 (filas)
 M = 2 (columnas)

2) Se podria realizar una mejora para que el algoritmo permita ejecutar operaciones y realizar comprobaciones de las propiedades con más de una matriz. Por ejemplo: la suma traspuesta de matrices es igual a la suma de las matrices traspuestas: (X+Z) EXP T = X EXP T + Z EXP T.

De esta manera el algoritmo podria servir aún más para fines educativos. 





---
# 6 Bibliografia

Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

Paginas de Referencia: [WEB]

https://stackoverrun.com/es/q/3720173

https://fisica.cab.cnea.gov.ar/gpgpu/images/clases/clase7_multiplicacion%20de%20matrices.pdf

https://developer.nvidia.com/blog/efficient-matrix-transpose-cuda-cc/


